**IRIS DATASET**

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

# Load dataset
data = load_iris()
X, y = data.data, data.target

# One-hot encode the target labels
encoder = OneHotEncoder(sparse_output=False)
y = encoder.fit_transform(y.reshape(-1, 1))

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**CREATE NEURAL NETWORK FROM SCRATCH**

In [ ]:
import numpy as np
class SimpleNN:
    def __init__(self, input_dim, hidden_dim, output_dim, learning_rate=0.01):
        # Initialize weights and biases
        self.W1 = np.random.randn(input_dim, hidden_dim)
        self.b1 = np.zeros((1, hidden_dim))
        self.W2 = np.random.randn(hidden_dim, output_dim)
        self.b2 = np.zeros((1, output_dim))
        self.learning_rate = learning_rate

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def sigmoid_derivative(self, z):
        return z * (1 - z)

    def forward(self, X):
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = self.sigmoid(self.z1)
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = self.sigmoid(self.z2)
        return self.a2

    def backward(self, X, y, output):
        error = output - y
        d_output = error * self.sigmoid_derivative(output)

        error_hidden = d_output.dot(self.W2.T)
        d_hidden = error_hidden * self.sigmoid_derivative(self.a1)

        self.W2 -= self.a1.T.dot(d_output) * self.learning_rate
        self.b2 -= np.sum(d_output, axis=0, keepdims=True) * self.learning_rate
        self.W1 -= X.T.dot(d_hidden) * self.learning_rate
        self.b1 -= np.sum(d_hidden, axis=0, keepdims=True) * self.learning_rate

    def train(self, X, y, epochs=1000):
        for _ in range(epochs):
            output = self.forward(X)
            self.backward(X, y, output)

    def predict(self, X):
        output = self.forward(X)
        return np.argmax(output, axis=1)

In [3]:
# Initialize the neural network
input_dim = X_train.shape[1]
hidden_dim = 10
output_dim = y_train.shape[1]
model = SimpleNN(input_dim, hidden_dim, output_dim, learning_rate=0.01)

# Train the model
model.train(X_train, y_train, epochs=10000)

# Inference
y_pred = model.predict(X_test)
y_true = np.argmax(y_test, axis=1)

# Evaluate accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy from Model: {accuracy * 100:.2f}%")

Accuracy from Model: 100.00%


**CREATE NEURAL NETWORK USING TENSORFLOW**

In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the model
tf_model = models.Sequential([
    layers.Input(shape=(input_dim,)),
    layers.Dense(hidden_dim, activation='sigmoid'),
    layers.Dense(output_dim, activation='softmax')
])

# Compile the model
tf_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
# Define a callback for saving the model
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint('tf_model.h5.keras', save_best_only=True)

# Train the model
tf_model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), callbacks=[checkpoint_callback])

Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 850ms/step - accuracy: 0.3529 - loss: 1.1240 - val_accuracy: 0.3667 - val_loss: 1.1268
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.2842 - loss: 1.1433 - val_accuracy: 0.3667 - val_loss: 1.1232
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3165 - loss: 1.1300 - val_accuracy: 0.3667 - val_loss: 1.1196
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3133 - loss: 1.1306 - val_accuracy: 0.3667 - val_loss: 1.1163
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.3342 - loss: 1.1182 - val_accuracy: 0.3667 - val_loss: 1.1130
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.2852 - loss: 1.1268 - val_accuracy: 0.3667 - val_loss: 1.1099
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3175 - loss: 1.1164 - val_accuracy: 0.3667 - val_loss: 1.1067
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.3060 - loss: 1.1147 - val_accuracy: 0.3667 - val_loss

In [8]:
# Load the best saved model
best_tf_model = tf.keras.models.load_model('tf_model.h5.keras')

# Evaluate the model
test_loss, test_acc = best_tf_model.evaluate(X_test, y_test, verbose=2)
print(f"Accuracy from TensorFlow Model: {test_acc * 100:.2f}%")

1/1 - 1s - 1s/step - accuracy: 0.9667 - loss: 0.5865
Accuracy from TensorFlow Model: 96.67%


**CREATE NEURAL NETWORK USING PYTORCH**

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Convert data to tensors
X_train_tensor = torch.Tensor(X_train)
y_train_tensor = torch.Tensor(np.argmax(y_train, axis=1)).long()
X_test_tensor = torch.Tensor(X_test)
y_test_tensor = torch.Tensor(np.argmax(y_test, axis=1)).long()

# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Define the model
class PyTorchNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(PyTorchNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.sigmoid = nn.Sigmoid()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.sigmoid(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model
pt_model = PyTorchNN(input_dim, hidden_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(pt_model.parameters(), lr=0.01)

In [10]:
# Define a callback to save the model
best_acc = 0.0
for epoch in range(100):
    pt_model.train()
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = pt_model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()

    # Evaluate on the test set
    pt_model.eval()
    with torch.no_grad():
        test_output = pt_model(X_test_tensor)
        _, y_pred_tensor = torch.max(test_output, 1)
        accuracy = (y_pred_tensor == y_test_tensor).sum().item() / len(y_test_tensor)

    # Save the best model
    if accuracy > best_acc:
        best_acc = accuracy
        torch.save(pt_model.state_dict(), 'pt_model.pth')

print(f"Accuracy from PyTorch Model: {best_acc * 100:.2f}%")

Accuracy from PyTorch Model: 100.00%
